# AvitoTech: Загрузка реальных данных из MinIO → CORE

**Источник**: `s3a://raw/avitotech/<dataset>/ingest_date=2026-02-14/`



In [35]:
# Инициализация Spark с поддержкой MinIO (S3)
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, coalesce, from_unixtime
from pyspark.sql.functions import broadcast 
from pyspark.sql.types import StructType, StructField, IntegerType, LongType, StringType, TimestampType, DateType
import psycopg2

spark = (SparkSession.builder 
    .appName("AvitoTech Real Data Load") 
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") 
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") 
    .config("spark.hadoop.fs.s3a.access.key", "airflow") 
    .config("spark.hadoop.fs.s3a.secret.key", "airflow123") 
    .config("spark.hadoop.fs.s3a.path.style.access", "true") 
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") 
    .config("spark.jars", "/jars/hadoop-aws-3.3.4.jar,/jars/aws-java-sdk-bundle-1.12.262.jar,/jars/postgresql-42.7.4.jar") 
    .config("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
    .config("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
    .config("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")
    .config("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")
    .config("spark.sql.parquet.outputTimestampType", "TIMESTAMP_MICROS")
    .config("spark.sql.parquet.timestampNTZ.enabled", "true")
    .config("spark.sql.legacy.parquet.nanosAsLong", "true")
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "100") \
    .config("spark.default.parallelism", "100") \
    .config("spark.memory.fraction", "0.7") \
    .config("spark.memory.storageFraction", "0.3")
    .getOrCreate()
)


print("Spark инициализирован")

Spark инициализирован


In [17]:
INGEST_DATE = "2026-02-14"
JDBC_URL = "jdbc:postgresql://postgres:5432/dwh"
JDBC_PROPS = {
    "user": "app",
    "password": "app",
    "driver": "org.postgresql.Driver"
}

In [36]:
# Загрузка данных из MinIO
# Схема для clickstream (с правильным типом event_date)
clickstream_schema = StructType([
    StructField("cookie", LongType(), True),
    StructField("item", LongType(), True),
    StructField("event", LongType(), True),
    StructField("event_date", TimestampType(), True),  # Критично: не datetime[ns], а TimestampType
    StructField("surface", LongType(), True),
    StructField("platform", LongType(), True),
    StructField("node", LongType(), True)
])

# Схема для events
events_schema = StructType([
    StructField("event", LongType(), True),
    StructField("is_contact", IntegerType(), True)  # 0/1 как целое число
])

# Схема для cat_features
cat_features_schema = StructType([
    StructField("item", LongType(), True),
    StructField("location", LongType(), True),
    StructField("category", LongType(), True),
    StructField("clean_params", StringType(), True),  # JSON-строка
    StructField("node", LongType(), True)
])

# Ячейка 3: Загрузка данных
df_click = spark.read \
    .option("datetimeRebaseMode", "CORRECTED") \
    .option("int96RebaseMode", "CORRECTED") \
    .parquet(f"s3a://raw/avitotech/clickstream/ingest_date={INGEST_DATE}/") \
    .repartition(50)

df_events = spark.read.parquet(f"s3a://raw/avitotech/events/ingest_date={INGEST_DATE}/")
df_cats = spark.read \
    .option("datetimeRebaseMode", "CORRECTED") \
    .option("int96RebaseMode", "CORRECTED") \
    .parquet(f"s3a://raw/avitotech/cat_features/ingest_date={INGEST_DATE}/") \
    .repartition(50)

# Получаем количество строк
click_count = df_click.count()
events_count = df_events.count()
cats_count = df_cats.count()

print(f"clickstream rows: {click_count:,}")
print(f"events rows: {events_count:,}")
print(f"cat_features rows: {cats_count:,}")

[Stage 330:=================================================>    (92 + 6) / 100]

clickstream rows: 68,806,152
events rows: 19
cat_features rows: 22,646,691


In [39]:
# Джойним события с флагом контакта и категориями/локациями
df_enriched = df_click \
    .join(broadcast(df_events), "event", "left") \
    .join(df_cats.select("item", "category", "location", "node"), "item", "left") \
    .withColumn("ingest_date", lit(INGEST_DATE).cast(DateType())) \
    .withColumn("event_date", (col("event_date") / 1000000000).cast("timestamp")) \
    .withColumn("is_contact", col("is_contact").cast("boolean"))

enriched_count = df_enriched.count()
print(f"Enriched facts: {enriched_count:,} rows")

[Stage 347:=====================================================>(99 + 1) / 100]

Enriched facts: 68,806,152 rows


In [40]:
# Идемпотентная загрузка в фактовую таблицу: удаляем старые данные за эту дату через psycopg2
conn = psycopg2.connect(
    host="postgres",
    database="dwh",
    user=JDBC_PROPS["user"],
    password=JDBC_PROPS["password"]
)
cur = conn.cursor()
cur.execute(f"DELETE FROM core.fct_user_interactions WHERE ingest_date = '{INGEST_DATE}'")
conn.commit()
cur.close()
conn.close()
print(f"Удалены старые записи за {INGEST_DATE}")

# Вставляем новые данные
df_to_insert = df_enriched.select(
    "cookie", "item", "event_date", "surface", "platform",
    "category", "location", "is_contact", "ingest_date"
).withColumnRenamed("surface", "surface_id") \
 .withColumnRenamed("platform", "platform_id") \
 .withColumnRenamed("category", "category_id") \
 .withColumnRenamed("location", "location_id")

# Проверим схему перед записью
print("Схема DataFrame перед записью в PostgreSQL:")
df_to_insert.printSchema()

# Запись в PostgreSQL
df_to_insert.write \
    .jdbc(JDBC_URL, "core.fct_user_interactions", mode="append", properties=JDBC_PROPS)

print("✅ Факты загружены в core.fct_user_interactions")


Удалены старые записи за 2026-02-14
Схема DataFrame перед записью в PostgreSQL:
root
 |-- cookie: long (nullable = true)
 |-- item: long (nullable = true)
 |-- event_date: timestamp (nullable = true)
 |-- surface_id: long (nullable = true)
 |-- platform_id: long (nullable = true)
 |-- category_id: long (nullable = true)
 |-- location_id: long (nullable = true)
 |-- is_contact: boolean (nullable = true)
 |-- ingest_date: date (nullable = true)



[Stage 365:====================================================> (98 + 2) / 100]

✅ Факты загружены в core.fct_user_interactions


In [47]:
# Финальная проверка
print("\n" + "="*60)
print("ЗАГРУЗКА ЗАВЕРШЕНА")
print("="*60)
print(f"Фактов загружено: {df_enriched.count():,}")


ЗАГРУЗКА ЗАВЕРШЕНА


[Stage 400:=====================================================>(99 + 1) / 100]

Фактов загружено: 68,806,152
